In [ ]:
#%matplotlib inline
import pandas as pd
from datetime import datetime, date
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.linear_model import LinearRegression,Ridge,BayesianRidge
from sklearn.cluster import Minibatchmeans
from sklearn.metrics import mean_squared_error
from math import radians, cos, sin, asin, sqrt
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams('figure.figsize') = [60,30]
from sklearn.cluster import KMeans

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, KFold

In [ ]:
train = pd.read_csv('train.csv',parse_dates = ['pickup_datetime'])
test = pd.read_csv('test.csv',parse_dates = ['pickup_datetime'])


In [ ]:
print('train shape',train.shape)
print('test shape', test.shape)

In [ ]:
print([i for i in train.columns if i not in test.columns])

holiday

In [ ]:
holiday = pd.read_csv('NYC_2016Holidays.csv',seq=';')
holiday.head()

In [ ]:
holiday['Date'] = holiday['Date'].apply(lambda x: x + ' 2016')
holiday.head()

In [ ]:
holidays = [datetime.strftime(holiday.loc[i,'Date'], '%B %d %Y').date() for i in range(len(holiday))]
holidays

routes from open ource routing machine

In [ ]:
fastrout1 = pd.read_csv('fastest_routes_train_part_1.csv',
                        usecols = ['id','total_distance','total_travel_time','number_of_steps','step_direction'])
fastrout2 = pd.read_csv('fastest_routes_train_part_2.csv',
                        usecols = ['id','total_distance','total_travel_time','number_of_steps','step_direction'])
fastrout = pd.concat((fastrout1,fastrout2))
fastrout.head()


In [ ]:
right_turn = []
left_turn = []
right_turn += list(map(lambda x:x.count('right') - x.count('slight right'),fastrout.step_direction)
left_turn += list(map(lambda x:x.count('left') - x.count('slight left'),fastrout.step_direction))

osrm_data = fastrout[['id','total_distance','total_travel_time','number_of_steps']]
osrm_data = osrm_data.assign(right_steps = right_turn)
osrm_data = osrm_data.assign(left_steps = left_turn)
osrm_data.head()

In [ ]:
osrm_test = pd.read_csv('fastest_rotes_test.csv')
right_turn = list(map(lambda x:x.count('right') - x.count('slight right'),osrm_test.step_direction)
left_turn = list(map(lambda x:x.count('left') - x.count('slight left'),osrm_test.step_direction))

osrm_test = osrm_test[['id','total_distance','total_travel_time','number_of_steps']]
osrm_test = osrm_test.assign(right_steps = right_turn)
osrm_test = osrm_test.assign(left_steps = left_turn)
osrm_test.head()

weather

In [ ]:
weather = pd.read_csv('KNYC_Motars.csv',paras_dates=['Time'])
weather.head()

Visualization

In [ ]:
lonngitude = list(train.pickup_longitude) + list(train.dropoff_longitude)
latitude =  list(train.pickup_latitude) + list(train.dropoff_latitude)

print(len(train.pickup_longitude),len(train.dropoff_longitude), len(lonngitude))
print(len(train.pickup_latitude),len(train.dropoff_latitude), len(latitude))

loc_df = pd.DataFrame()
loc_df['longitude'] = lonngitude
loc_df['latitude'] = latitude
loc_df.head()


In [ ]:
xlim = [-74.03,-73.77]
ylim = [40.63,40.85]

print(loc_df.shape)
loc_df = loc_df[(loc_df.longtitude > xlim[0]) & (loc_df.longtitude < xlim[1])]
loc_df = loc_df[(loc_df.longtitude > xlim[0]) & (loc_df.longtitude < xlim[1])]
loc_df = loc_df[(loc_df.latitude > ylim[0]) & (loc_df.latitude < ylim[1])]
loc_df = loc_df[(loc_df.latitude > ylim[0]) & (loc_df.latitude < ylim[1])]
print(loc_df.shape)


In [ ]:
kmeans = KMeans(n_clusters=15, random_state=2, n_init=10).fit(loc_df)
loc_df['label'] = kmeans.labels_
loc_df.head()

In [ ]:
plt.figure(figsize=(10,10))
for label in loc_df.label.unique():
    plt.plot(loc_df.longtitude[loc_df.label == label],loc_df.latitude[loc_df.label == label],'.',alpha = 0.3,markersize=)

plt.title('Cluster of New York')
plt.show()

Feature engineering

date feature,including holldays

In [ ]:
print('train shape',train.shape)
print('test shape',test.shape)

In [ ]:
for df in (train,test):
    df['year'] = df[pickup_datatime].dt.year
    df['month'] = df[pickup_datatime].dt.month
    df['day'] = df[pickup_datatime].dt.day
    df['hr'] = df[pickup_datatime].dt.hr
    df['minute'] = df[pickup_datatime].dt.minute
    df['store_and_fwd_flag'] = 1 * (df.store_and_fwd_flag.values =='Y')

In [ ]:
train = train.assign(log_trip_duration - np.log(train.trip_duration+1))

In [ ]:
time_data = pd.DataFrame(index = range(len(train)))
time_test = pd.DataFrame(index = range(len(test)))

In [ ]:
def restday(yr,month,day,holidays):
    '''
    output:
        is_rest: a list of Boolean variable indicating if the sample occurred in the rest day.
        is_weekend: a list of Boolean variable indicating if the sample occurred in the weekend.
    '''
    is_rest     =[None]*len(yr)
    is_weekend     =[None]*len(yr)
    i = 0
    for yy,mm,dd in zip(yr,month,day):
        is_weekend[i] = date(yy,mm,dd).isoweekday() in (6,7)
        is_rest[i] = is_weekend[i] or date(yy,mm,dd) in holidays
        i+=1
    return is_rest,is_weekend

In [ ]:
rest_day,weekend = testday(train.year,train.month,train.day,holidays)
train['rest_day'] = rest_day
train['weekend'] = weekend

rest_day,weekend = restday(test.year,test.month,test.day,holidays)
test['rest_day'] = rest_day
test['weekend'] = weekend

train['pickup_time'] = train.hr+train.minute/60
test['pickup_time'] = test.hr+test.minute/60

In [ ]:
for df in (train,test):

    df['hr_categori'] = np.nan
    df.loc[(df.rest_day == False) & (df.hr <= 9)&(df.hr >= 7), 'hr_categori'] = "rush"
    df.loc[(df.rest_day == False) & (df.hr <= 18)&(df.hr >= 16), 'hr_categori'] = "rush"
    df.loc[(df.rest_day == False) & (df.hr <= 16)&(df.hr >= 9), 'hr_categori'] = "day"
    df.loc[(df.rest_day == False) & (df.hr <= 7)&(df.hr >= 18), 'hr_categori'] = "night"
    df.loc[(df.rest_day == True) & (df.hr <= 18)&(df.hr >= 7), 'hr_categori'] = "day"
    df.loc[(df.rest_day == True) & (df.hr <= 7)&(df.hr >= 10), 'hr_categori'] = "night"


In [ ]:
#11
print('train shape',train.shape)
print('test shape', test.shape)

In [ ]:
print([i for i in train.columns if i not in test.columns])

distance feature

In [ ]:
train = train.join(osrm_data.set_index('id'),on='id')

In [ ]:
test = test.join(osrm_test.set_inndex('id'),on='id')

In [ ]:
print('train shape',train.shape)
print('test shape',test.shape)

weather feature

In [ ]:
weather['snow'] = 1 * (weather.Events == 'Snow') + 1 * (weather.Events=='fog\n\t,\nSnow')
weather['year'] = weather['Time'].dt.year
weather['month'] = weather['month'].dt.month
weather['day'] = weather['day'].dt.day
weather['hr'] = weather['hr'].dt.hour
weather = weather[weather['year'] == 2016]['month','day','hr','temp','Precip','snow','Visibility']
print(weather.shape)
weather.head()

In [ ]:
train = pd.merge(train, weather, on = ['month','day','hr'],how = left)
test = pd.merge(test, weather, on = ['month','day','hr'],how = left)

In [ ]:
print('train shape',train.shape)
print('test shape',test.shape)

clustering and find speed

In [ ]:
coords = np.vstack((train[['pickup_latiyude','pickip_longitude']],values,
                   train[['dropoff_latiyude','dropoff_longitude']],values,
                    test[['pickup_latiyude','pickip_longitude']],values,
                    test[['dropoff_latiyude','dropoff_longitude']],values,))
coords

In [ ]:
sample_ind = np.random.permutation(len(coords))[:1000000]
kmeans = MiniBatchKMeans(n_cluster-8, batch_size=10000).fit(coords[sample_ind])